In [ ]:
import mbuild as mb
import foyer
import unyt as u

# Prepare recipes with mBuild - UA polymer


We will be creating a united atom polymer (i.e., no-hydrogen atoms) that we will atom-type with using the TraPPE force field.  

To create this polymer we need two different interaction sites, one corresponding to CH3 (terminal interaction sites) and one corresponding to CH2 (middle interactions sites). 

Let us first examine the code to create a CH3 site:

In [ ]:
class _CH3(mb.Compound):
    def __init__(self):
        super().__init__()
        self.name = "_CH3"
        
        """Create a particle name _CH3 and add it to self."""
        _ch3 = mb.Compound(pos=[0.0, 0.0, 0.0], name='_CH3')
        self.add(_ch3)
     
        """Create a port attached to the _CH3 particles, label as 'up'.
        Use the separation value of the _CH2's ports as above.
        """
        port_up = mb.Port(anchor=_ch3, orientation=[0, 1, 0], separation=0.07)
        self.add(port_up, "up")         

Create a temporary instance of the `_CH3` compound and visualize it.  Note we tell the visualizer to render the ports so we can check to provide visual confirmation that we have done things properly.

In [ ]:
temp_compound = _CH3()
temp_compound.visualize(backend='nglview', show_ports=True)

### Exercise 1 - Create your the CH2 building block

Using the `_CH3` class above as an example, fill in the code block below to create a CH2 with two ports diametrically opposed.

In [ ]:
class _CH2(mb.Compound):
    def __init__(self):
        super().__init__()
        self.name = "_CH2"
        """Create a particle name _CH2 and add it to self."""
        # Enter your code here

        
        """Create a port attached to the _CH2 particles, label as 'up'."""
        # Enter your code here


        
        
        """Create a port attached to the _CH2 particles, label as 'down',
        with same separation but opposite orientation as 'up port'
        """
        # Enter your code here


        

In [ ]:
# instantiate a temporary instance of _CH2 and visualize it showing the ports 



### Exercise 2a - Create the "recipe" 

In [ ]:
class CG_Alkane(mb.Compound):
    def __init__(self, n):
        super().__init__()
        self.name = "CG_Alkane"
    
        if n == 1: 
            """Create a _CH4 bead and add it to self."""
            _ch4 = mb.Compound(pos=[0.0, 0.0, 0.0], name='_CH4')
            self.add(_ch4)

        elif n == 2:
            """Create 2 _CH3 beads and bonds them together, then add both to self."""
            _ch3a = _CH3()
            self.add(_ch3a, "_CH3_a")
            
            """Create the other _CH3 bead."""
            _ch3b = _CH3()
            self.add(_ch3b, "_CH3_b")
            
            """Force overlap ports of the two _CH3 beads we just created"""
            mb.force_overlap(move_this=_ch3b, 
                             from_positions=_ch3b['up'],
                             to_positions=_ch3a['up'])
        else:
            """ Create a _CH3 particle and add it to self with label 'link0'"""
            # this will keep track of particle id as we add it to the system
            pid = 0
            _ch3top =_CH3()
            self.add(_ch3top, f"link{pid}")


            for i in range(n - 2):
                """ Create a _CH2 particle and add it to self with label f'link{pid}'.
                Then force_overlap it (from Port "down") with 
                the previous link (to Port "up").
                """
                _ch2 = _CH2()
                self.add(_ch2, f"link{pid+1}")
                
                """Force overlap the 'down' port of the _ch2 we just created
                to 'up' port of the last link of the alkane.
                """
                # Enter your code
                mb.force_overlap(move_this=,
                                 from_positions=, 
                                 to_positions=self[f"link{pid}"]["up"])
                
                #iterate the pid
                pid = pid+1             


            """ Create another _CH3 particle and add it to self with label f'link{i+2}.
            Then force_overlap it with the previous link f"link{i+1}" (note that .
            """
            _ch3bot =_CH3()
            self.add(_ch3bot, f"link{pid+1}")

            # Enter your code here
            mb.force_overlap(move_this=,
                 from_positions=, 
                 to_positions=  )


In [ ]:
test = CG_Alkane(n=5)
test.visualize()

### Exercise 2b - A shortcut with built-in Polymer builder

In [ ]:
from mbuild.lib.recipes import Polymer

"""Create a Polymer() object with _CH2 monomer"""
temp_polymer = Polymer(monomers=[_CH2()] )
"""Build the just-created object with length 2"""
temp_polymer.build(n=2)


In [ ]:
temp_polymer.visualize(backend="nglview", show_ports=True)

In this example, we do not have the `_CH3` groups capping the ends of the polymer.  To include these these capping groups, you can pass the Polymer class a list of two capping Compounds.  

### Use the help function to see the syntax, and modify the code above to include the `_CH3` capping groups and revisualize.

In [ ]:
help(Polymer)

### Exercise 3 - Create a box of pentane

Next, we will fill a box with pentane molecules.  

To do this we will create an instance of the polymer class, define a box, and then call the fill_box routine that uses PackMol. 

#### Hint: try call help(mb.packing.fill_box) to see the syntax to use

In [ ]:
help(mb.packing.fill_box)

In [ ]:
"""Create a pentane using the CG_Alkane recipe above."""
# Enter your code here
n_pentane = CG_Alkane(n=5)

"""Create a box of pentane using the mb.packing.fill_box() method.
Let's fill the box of size [4, 4, 4] with a edge padding of 0.2, i.e., edge=0.2 
with 150 compounds."""

box = mb.Box(lengths=[4,4,4])

# Hint: try call help(mb.packing.fill_box) in a separate cell
# Enter your code here
pentane_box = mb.packing.fill_box()


# Atomtyping and Parameterization with Foyer
### Exercise 4 - Loading the forcefield and perform atomtyping

Here we will load the trapp-ua forcefield and apply it to the box of pentane we just initialized above.

In [ ]:
import foyer 
from foyer import Forcefield

"""Load the TRAPPE-UA forcefield (name="trappe-ua")"""
trappe = foyer.Forcefield(name="trappe-ua")

"""Use foyer to apply forcefields to a pentane and a box of pentane"""
structure = trappe.apply(pentane_box)


In [ ]:
help(trappe.apply)

# Run Simulations
### Exercise 5a - Simulation with GROMACS

#### Saving the atom-typed structure to file
We have now created the various MDP files for gromacs.  We still need to write out our atom-typed system to the correspdonding GROMACS format.

In [ ]:
structure.save("gmx_sim/init.top", overwrite=True)
structure.save("gmx_sim/init.gro", overwrite=True)

#### Running GROMACS 
Note this is just a short run to demonstrate that it works; simulation data associated with a longer run is included for visualization later.

In [ ]:
%cd gmx_sim
!gmx grompp -f em.mdp -o em.tpr -c init.gro -p init.top --maxwarn 1
!gmx mdrun -v -deffnm em -s em.tpr -cpi em.cpt

!gmx grompp -f nvt.mdp -o nvt.tpr -c em.gro -p init.top --maxwarn 1
!gmx mdrun -v -deffnm nvt -s nvt.tpr -cpi nvt.cpt

!gmx grompp -f npt.mdp -o npt.tpr -c nvt.gro -p init.top --maxwarn 1
!gmx mdrun -v -deffnm npt -s npt.tpr -cpi npt.cpt
%cd ..

### Exercise 5b - Simulation with HOOMD-Blue

In [ ]:
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield
import hoomd 

# ref_distance: 10 angstrom -> 1 nm
# ref_energy: 1/4.184 kcal/mol -> 1 kJ/mol
# ref_mass: 0.9999938574 dalton -> 1 amu
d = 10
e = 1 / 4.184
m = 0.9999938574

"""Converting a typed structure to hoomd forcefield and snapshot"""
snapshot, forcefield, ref_vals = create_hoomd_forcefield(
    structure,
    ref_distance=d,
    ref_energy=e,
    ref_mass=m,
    r_cut=1,
    init_snap=None,
    pppm_kwargs={"Nx": 64, "Ny": 64, "Nz": 64, "order": 7},
)

for force in forcefield:
    if isinstance(force, hoomd.md.pair.LJ):
        force.tail_correction = True
        
forcefield[0].nlist.exclusions = ["bond", "1-3", "1-4"]

"""Starting up the system and adding in operations and loggers"""
device = hoomd.device.auto_select()
sim = hoomd.Simulation(device=device)
sim.create_state_from_snapshot(snapshot)

gsd_writer = hoomd.write.GSD(
    filename=f"hoomd_sim/trajectory.gsd",
    trigger=hoomd.trigger.Periodic(10000),
    mode="wb",
    dynamic=["momentum"],
)
sim.operations.writers.append(gsd_writer)

logger = hoomd.logging.Logger(categories=["scalar", "string"])
logger.add(sim, quantities=["timestep", "tps"])
thermo_props = hoomd.md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())
sim.operations.computes.append(thermo_props)
logger.add(
    thermo_props,
    quantities=[
        "kinetic_energy",
        "potential_energy",
        "pressure",
        "kinetic_temperature",
        "volume",
    ],
)

table_file = hoomd.write.Table(
    output=open(
        "hoomd_sim/log.txt", mode="w", newline="\n"
    ),
    trigger=hoomd.trigger.Periodic(period=1000),
    logger=logger,
    max_header_len=7,
)
sim.operations.writers.append(table_file)

"""Create integrators and add them to the simulation context, sequentially"""
dt = 0.001
integrator = hoomd.md.Integrator(dt=dt)
integrator.forces = forcefield

kT = (372 * u.K).to_equivalent("kJ/mol", "thermal").value

tau = 100 * dt 
tauS = 1000 * dt 

pressure = (14.02 * u.kPa).to("kJ/(mol*nm**3)").value

"""Run NVT"""
nvt_steps = 1e6

nvt_method = hoomd.md.methods.NVT(filter=hoomd.filter.All(),
                                         kT=kT,
                                         tau=tau)

integrator.methods = [nvt_method]

sim.operations.integrator = integrator
sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), 
                                      kT=kT)

sim.run(nvt_steps)

"""Run NPT"""
npt_steps = 1e6
npt_method = hoomd.md.methods.NPT(filter=hoomd.filter.All(),
                                 kT=kT, 
                                 tau=tau,
                                 S=pressure,
                                 tauS=tauS,
                                 couple="xyz")
integrator.methods = [npt_method]
sim.run(1e6)

### Exercise 6a - Data analysis of GROMACS simulation

In [ ]:
"""If you used GROMACS"""
import numpy as np
import pylab as plt 

import panedr
from panedr import edr_to_df

"""In the interest of time, loading in pre-simulated data"""
data = edr_to_df("../master_notebooks/example_workflow/gmx_sim/npt.edr")

plt.rcParams['font.family'] = "DIN Alternate"
font = {'family' : 'DIN Alternate',
        'weight' : 'normal',
        'size'   : 12}

fig, ax = plt.subplots(1, 1)

ax.spines["bottom"].set_linewidth(3)
ax.spines["left"].set_linewidth(3)
ax.spines["right"].set_linewidth(3)
ax.spines["top"].set_linewidth(3)

ax.title.set_text('Control plot')
ax.set_xlabel(r"MD Step")
ax.set_ylabel('Density $(kg / m{^3})$')
ax.yaxis.tick_left()
ax.yaxis.set_label_position('left')
ax.axhline(y=541, color='r', linestyle='-', label='~TraPPE-UA Density')

dt, density = list(), list()
for i, j in enumerate(data["Density"]):
    dt.append(i)
    density.append(j)
    
ax.plot(dt, density, "-", color='lightgray', label='Density')
ax.legend(loc="best")
plt.show()

### Exercise 6b - Data analysis of HOOMD-Blue simulation


In [ ]:
import numpy as np
import pylab as plt 

"""In the interest of time, loading in pre-simulated data"""
data = np.genfromtxt("../master_notebooks/example_workflow/hoomd_sim/log.txt", names=True)
system_mass = 72.15 * u.amu * 150 
volume = data["volume"] * u.nm**3 

density = (system_mass / volume).to("kg/m**3")
plt.rcParams['font.family'] = "DIN Alternate"
font = {'family' : 'DIN Alternate',
        'weight' : 'normal',
        'size'   : 12}

fig, ax = plt.subplots(1, 1)

ax.spines["bottom"].set_linewidth(3)
ax.spines["left"].set_linewidth(3)
ax.spines["right"].set_linewidth(3)
ax.spines["top"].set_linewidth(3)

ax.title.set_text('Control plot')
ax.set_xlabel(r"MD Step")
ax.set_ylabel('Density $(kg / m{^3})$')
ax.yaxis.tick_left()
ax.yaxis.set_label_position('left')
ax.axhline(y=541, color='r', linestyle='-', label='~TraPPE-UA Density')

    
ax.plot(data["timestep"][1000:], density[1000:], "-", color='lightgray', label='Density')
ax.legend(loc="best")
plt.show()